In [1]:
#default_exp utils.math.preprocessing

In [2]:
import sys
sys.path.insert(0, "../pointpillars")

from utils.io import read_config
from utils.time import time_method


In [3]:
#export
import torch
import numpy as np
import numba
from numba import njit


In [4]:
#export
class Preprocessing(object):

    def __init__(self, pillars_cfg):
        self.x_min = float(pillars_cfg["x_min"])
        self.y_min = float(pillars_cfg["y_min"])
        self.z_min = float(pillars_cfg["z_min"])
        self.x_max = float(pillars_cfg["x_max"])
        self.y_max = float(pillars_cfg["y_max"])
        self.z_max = float(pillars_cfg["z_max"])
        self.x_step = float(pillars_cfg["x_step"])
        self.y_step = float(pillars_cfg["y_step"])
        self.max_ppp = pillars_cfg.getfloat("max_points_per_pillar")
        self.max_pil = pillars_cfg.getint("max_pillars")
        self.nb_pil_x = int((self.x_max - self.x_min) / self.x_step)
        self.nb_pil_y = int((self.y_max - self.y_min) / self.y_step)
        self.nb_pil = self.nb_pil_x * self.nb_pil_y
        
    def remove_invalid_points(self, pcloud):
        return Preprocessing._remove_invalid_points(pcloud, 
                                                   self.x_min, 
                                                   self.x_max, 
                                                   self.y_min, 
                                                   self.y_max, 
                                                   self.z_min, 
                                                   self.z_max)

    @staticmethod
    @njit("f4[:,:](f4[:,:],f4, f4, f4, f4, f4, f4)")
    def _remove_invalid_points(pcloud: np.ndarray, 
                               x_min: float,
                               x_max: float,
                               y_min: float,
                               y_max: float,
                               z_min: float,
                               z_max: float):
        """
        Remove invalid points from the point cloud outside of given
        the given range

        :param pcloud: ndarray(nb_points, 4) with all points from point cloud
        :returns: ndarray(nb_points, 4) with all points inside bounds
        """
        # reshape here to get njit improvements
        x = pcloud[:,0]
        y = pcloud[:,1]
        z = pcloud[:,2]
        cond_x = np.logical_and(x >= x_min, x <= x_max)
        cond_y = np.logical_and(y >= y_min, y <= y_max)
        cond_z = np.logical_and(z >= z_min, z <= z_max)
        cond_xy = np.logical_and(cond_x, cond_y)
        cond = np.logical_and(cond_xy, cond_z)
        indices = np.where(cond)[0]
        #pcloud = np.delete(pcloud, indices)
        pcloud = pcloud[indices,:]

        return pcloud
    
    def get_points_in_pillars(self, pcloud):
        """
        :returns: (pill_tens(max_pil, max_ppp, 4)
        """
        pillars = np.zeros([self.nb_pil, int(self.max_ppp), 4], dtype=np.float32)
        pill_ind = np.zeros([self.nb_pil, 2], dtype=np.float32)
        pill_point_nbr = np.zeros([self.nb_pil], dtype=np.int32)
        coor_to_pillar_id = -1 * np.ones([self.nb_pil_x, self.nb_pil_y], dtype=np.int32)

        min = np.array((self.x_min, self.y_min), dtype=np.float32)
        step = np.array((self.x_step, self.y_step), dtype=np.float32)


        Preprocessing._get_points_in_pillars(pcloud,
                                             pillars,
                                             pill_ind,
                                             pill_point_nbr,
                                             coor_to_pillar_id,
                                             min,
                                             step,
                                             self.max_ppp)

        pill_tens = pillars[:self.max_pil]
        pill_ind_tens = pill_ind[:self.max_pil]
        
        return pill_tens, pill_ind_tens
        

    @staticmethod
    @njit(parallel=False)
    def _get_points_in_pillars(pcloud: np.ndarray,
                               pillars : np.ndarray,
                               pill_ind: np.ndarray,
                               pill_point_nbr: np.ndarray,
                               coor_to_pillar_id: np.ndarray,
                               min: np.ndarray,
                               step: np.ndarray,
                               max_ppp: int):
        """
            pcloud[nbpoints, 4]: array with points from point cloud (should be shuffled)
            pillars[nbpillars, 50, 4]: array with the final pcloud points per pillar (init as zero)
            pill_ind[nbpillars, 2]: x,y index for each pillar (init as zero)
            pill_point_nbr[nbpillars]: non zero points currently in the pillar
            coor_to_pillar_id[x_nbr, y_nbr, 1]: with the xy bounds from the pillar get the pillar id (init with -1)

            returns: number of non zero pillars

        """
        N = pcloud.shape[0]
        pill_num = 0
        for i in range(N):
            point = pcloud[i]
            pil_coor = ((point[:2] - min) / step).astype(np.int32)
            pillar_id = coor_to_pillar_id[pil_coor[0], pil_coor[1]]
            # the pillar is not used yet
            if pillar_id == -1:
                pillar_id = pill_num
                coor_to_pillar_id[pil_coor[0], pil_coor[1]] = pillar_id
                pill_num += 1
                # set the correct bound so they can be transformed
                pill_ind[pillar_id] = pil_coor

            # non zero points in current pillar
            p_nbr = pill_point_nbr[pillar_id]
            if p_nbr < max_ppp:
                pillars[pillar_id, p_nbr] = point
                pill_point_nbr[pillar_id] += 1

        return pill_num

    def __call__(self, pcloud: np.ndarray, shuffle: bool = True):
        """
            not shuffling saves about 2ms, takes around 20-21ms
        """
        pcloud = self.remove_invalid_points(pcloud)
        if shuffle:
            np.random.shuffle(pcloud)


        return self.get_points_in_pillars(pcloud)

In [5]:
pillars_cfg = read_config()['pillars']
prepro = Preprocessing(pillars_cfg)
pcloud = np.random.rand(140000, 4) * 100

out = prepro.remove_invalid_points(pcloud)

In [6]:
time_method(prepro.remove_invalid_points, runs=100, kwargs={"pcloud": pcloud})

tensor(1.0664)

In [7]:
out = prepro(pcloud)

In [8]:
time_method(prepro.__call__, runs=100, kwargs={"pcloud": pcloud, "shuffle": True})

tensor(3.8026)

In [9]:
time_method(prepro.__call__, runs=100, kwargs={"pcloud": pcloud, "shuffle": False})

tensor(2.7117)